**DTS - Complete Streets** *Right-of-Way Widths for Planned Street Improvements*



# 09 - Environmental Analysis

**Author:** rmangan

---

**Purpose:**

Calculate metrics related to Economic Justice and Sea Level Rise Exposure Area 

**This script performs the following functions:**

1. Add and calculate fields to indicate length and percent of segment inside Economic Justice polygons

2. Add and calcualte fields to indicate length and percent of segment inside Sea Level Rise Exposre Area polygons


**Global Assumptions and Notes:**
1. Economic Justice Dataset (requires written request to Oahu MPO)
    1. Homepage = https://www.oahumpo.org/projects/planning-studies/title-vi-and-environmental-justice-monitoring/
2. Sea Level Rise Exposure Area Dataset
    1. Metadata = https://files.hawaii.gov/dbedt/op/gis/data/slr_exposure_area_3_pt_2_ft.html
    2. Data  = https://files.hawaii.gov/dbedt/op/gis/data/slr_exposure_area_all.shp.zip

**Non-Standard Python Modules utilized:**
1. arcpy 2.7 - used for geo-processing

In [ ]:
# import modules
import arcpy
import os

In [ ]:
# set environment setttings
arcpy.env.workspace = "Z:\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\modal\Modal_Composite4.gdb"
arcpy.env.overwriteOutput = True

In [ ]:
# define variables
input_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb"

scratch_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_scratch.gdb"

output_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\scratch_GDBs\modal_composite_output.gdb"


# Input Datasets

modal_composite = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb\modal_composite_05_3"

EJ = r"Z:\H\Honolulu_DTS\D3409300_RailActivation\Transfer\Communications\From\External\20211005_DTS_T6EJ\T6EJ_Data.gdb\OahuMPO_EJ_Areas_Updated_2016_Block_Groups"

SLR = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb\SLR"

In [ ]:
def percent(input_dataset, poly_dataset, new_length_field, new_length_alias, new_percent_field, new_percent_alias):
    #take an input line dataset and calculate length and  percentage by length that each line segment
    #inersects with a polygon dataset
    #this function adds 2 new fields to the input dataset  
    
    
    #create lyr for analysis
    print("create analysis lyr...")
    lyr = arcpy.management.MakeFeatureLayer(poly_dataset, "lyr", where_clause = "")
    
    #dissolve polygon lyr to single feature
    print("dissolving polygon data...")
    dissolve_output = os.path.join(scratch_gdb_path, "dissolve")
    dissolve = arcpy.management.Dissolve(lyr, dissolve_output)
    
    #intersect line data w/ polygon dissolve output
    #output is lines that are inside polygon areas
    print("intersecting polygon data with lines...")
    intersect_output = os.path.join(scratch_gdb_path, "intersect")
    intersect = arcpy.analysis.Intersect([input_dataset, dissolve], intersect_output)
    
    #add length field to intersection output if it doesnt already exist
    print("add {} field...".format(new_length_field))
     
    list_fields = arcpy.ListFields(intersect)
    field_names = [i.name for i in list_fields]

    if new_length_field in field_names:
        print("{} field already exists".format(new_length_field))
    else:  
        print("adding field...")
        arcpy.AddField_management(intersect,
                                  field_name = new_length_field,
                                  field_type="FLOAT",
                                  field_alias = new_length_alias)
        
    #calc length field from shape.length with update cursor
    print("calc field from shape.length...")
    with arcpy.da.UpdateCursor(intersect,["Shape_Length", new_length_field]) as cursor:
        for row in cursor:
            try:
                row[1] = row[0]        
                cursor.updateRow(row)    
            except ValueError as error:
                print(error)
                
    #run frequency on output and sum length fields to normalize data by SegmentID
    #accounts for lines that may have been split by intersect
    print("summarize intersect table by SegmentID...")
    freq_output = os.path.join(scratch_gdb_path, "freq_output")
    freq = arcpy.analysis.Frequency(intersect, freq_output, "SEGMENTID", new_length_field)
    
    #join freq result back to modal comp input dataset
    print("Join field back to input on SegmentID...")
    join_target = input_dataset
    join_target_field = "SEGMENTID"
    join_table = freq
    join_table_field = "SEGMENTID"
    join_fields = [new_length_field]
    
    arcpy.JoinField_management(join_target, join_target_field, join_table, join_table_field, join_fields)
    
    #add new field to input dataset for length % if it doesn't already exist
    print("add new field for length percentage...")
    list_fields = arcpy.ListFields(input_dataset)
    field_names = [i.name for i in list_fields]

    if new_percent_field in field_names:
        print("{} field already exists".format(new_percent_field))
    else:
        print("adding percent field...")
        arcpy.AddField_management(modal_composite,
                                  field_name = new_percent_field,
                                  field_type="FLOAT",
                                  field_alias = new_percent_alias)

    #calculate line percentages based on length differences
    #calc field from shape.length with update cursor
    print("calc percent field...")
    with arcpy.da.UpdateCursor(input_dataset,[new_length_field,"Shape_Length",new_percent_field]) as cursor:
        for row in cursor:
            try:
                if row[0] is not None:
                    row[2] = row[0]/row[1]  
                    
                cursor.updateRow(row)    
            except ValueError as error:
                print(error)
    

## Perform Analysis

In [ ]:
##Compute Economic Justice area metrics
new_field = "EJ_percent"
new_alias = "EJ % by length"
percent(modal_composite, EJ, new_field, new_alias):

##Compute Sea Level Rise Exposure Area metrics
new_field = "SLR_percent"
new_alias = "SLR % by length"
percent(modal_composite, EJ, new_field, new_alias):

    
print("processing complete")